### Implementing FunkSVD - Solution

In this notebook we will take a look at writing our own function that performs FunkSVD, which will follow the steps you saw in the previous video.  If you find that you aren't ready to tackle this task on your own, feel free to skip to the following video where you can watch as I walk through the steps.

To test our algorithm, we will run it on the subset of the data you worked with earlier.  Run the cell below to get started.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

# avoiding kernel troubles
reviews = reviews.head(300000)


del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
print(ratings_mat)

[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]]


`1.` You will use the **user_movie_subset** matrix to show that your FunkSVD algorithm will converge.  In the below cell, use the comments and document string to assist you as you complete writing your own function to complete FunkSVD.  You may also want to try to complete the funtion on your own without the assistance of comments.  You may feel free to remove and add to the function in any way that gets you a working solution! 

**Notice:** There isn't a sigma matrix in this version of matrix factorization.

In [2]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0] # number of rows in the matrix
    n_movies = ratings_mat.shape[1] # number of mvies in the matrix
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat)) # total number of ratings in the matrix
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # keep track of iteration and MSE
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` Try out your function on the **user_movie_subset** dataset.  First try 4 latent features, a learning rate of 0.005, and 10 iterations.  When you take the dot product of the resulting **U** and **V** matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?

In [3]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 44.363526
2 		 23.775260
3 		 9.175984
4 		 3.552208
5 		 2.258471
6 		 2.017241
7 		 1.951357
8 		 1.911628
9 		 1.875676
10 		 1.839555


In [4]:
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[10.29695367  8.10907688 11.23622352  9.66476852]
 [ 9.17628942  6.38974161  9.70151852  8.53376262]
 [ 8.09374533  6.36037277  8.74854471  7.66707919]
 [10.02223118  7.30528421 10.46931912  9.26152785]
 [ 9.11874455  6.35802746  9.48763286  8.40941565]
 [ 7.14109808  5.19286929  7.66991813  6.73114606]
 [ 9.61340243  6.88810718 10.16260191  9.02719426]
 [ 8.76597132  6.3937644   9.07395761  8.03207213]]
[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]]


**The predicted ratings from the dot product are already starting to look a lot like the original data values even after only 10 iterations.  You can see some extreme low values that are not captured well yet.  The 5 in the second to last row in the first column is predicted as an 8, and the 4 in the second row and second column is predicted to be a 7.  Clearly the model is not done learning, but things are looking good.**

`3.` Let's try out the function again on the **user_movie_subset** dataset.  This time we will again use 4 latent features and a learning rate of 0.005.  However, let's bump up the number of iterations to 250.  When you take the dot product of the resulting **U** and **V** matrices, how does the resulting **user_movie** matrix compare to the original subset of the data?  What do you notice about your error at the end of the 250 iterations?

In [5]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 46.417626
2 		 25.712635
3 		 9.987626
4 		 3.607649
5 		 2.116413
6 		 1.839248
7 		 1.759083
8 		 1.707180
9 		 1.659216
10 		 1.610894
11 		 1.561161
12 		 1.509792
13 		 1.456880
14 		 1.402674
15 		 1.347525
16 		 1.291847
17 		 1.236105
18 		 1.180794
19 		 1.126411
20 		 1.073440
21 		 1.022327
22 		 0.973458
23 		 0.927144
24 		 0.883613
25 		 0.842998
26 		 0.805345
27 		 0.770615
28 		 0.738700
29 		 0.709431
30 		 0.682598
31 		 0.657966
32 		 0.635286
33 		 0.614310
34 		 0.594800
35 		 0.576537
36 		 0.559323
37 		 0.542987
38 		 0.527383
39 		 0.512391
40 		 0.497916
41 		 0.483886
42 		 0.470247
43 		 0.456964
44 		 0.444016
45 		 0.431392
46 		 0.419094
47 		 0.407128
48 		 0.395506
49 		 0.384243
50 		 0.373357
51 		 0.362864
52 		 0.352781
53 		 0.343121
54 		 0.333898
55 		 0.325120
56 		 0.316793
57 		 0.308919
58 		 0.301498
59 		 0.294523
60 		 0.287988
61 		 0.281882
62 		 0.276192
63 		 0.270901
64 		

In [6]:
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[10.02496399 10.0027939   9.99819745  9.98151491]
 [ 9.97926009  3.9976154   9.00133496 10.01517905]
 [ 7.95861661  8.99516515 10.00223899  5.03097028]
 [ 8.98646496  7.99906853 10.00218035 10.00949684]
 [10.007406    5.00079407  8.99950833  8.99500216]
 [ 6.03374779  4.00409194  9.99846406  5.97477924]
 [ 9.03595726  8.00387999  9.99771999  8.97328201]
 [ 9.96801515  4.99622666  9.00113524  8.02418786]]
[[10. 10. 10. 10.]
 [10.  4.  9. 10.]
 [ 8.  9. 10.  5.]
 [ 9.  8. 10. 10.]
 [10.  5.  9.  9.]
 [ 6.  4. 10.  6.]
 [ 9.  8. 10.  9.]
 [10.  5.  9.  8.]]


**In this case, we were able to completely reconstruct the item-movie matrix to obtain an essentially 0 mean squared error. I obtained 0 MSE on iteration 165.**

The last time we placed an **np.nan** value into this matrix the entire svd algorithm in python broke.  Let's see if that is still the case using your FunkSVD function.  In the below cell, I have placed a nan into the first cell of your numpy array.  

`4.` Use 4 latent features, a learning rate of 0.005, and 250 iterations.  Are you able to run your SVD without it breaking (something that was not true about the python built in)?  Do you get a prediction for the nan value?  What is your prediction for the missing value? Use the cells below to answer these questions.

In [7]:
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.]])

In [8]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 52.083106
2 		 34.164582
3 		 16.735618
4 		 6.755619
5 		 3.231859
6 		 2.263011
7 		 1.972706
8 		 1.852647
9 		 1.782152
10 		 1.728268
11 		 1.679874
12 		 1.632603
13 		 1.584614
14 		 1.535146
15 		 1.483957
16 		 1.431082
17 		 1.376727
18 		 1.321208
19 		 1.264922
20 		 1.208319
21 		 1.151882
22 		 1.096101
23 		 1.041454
24 		 0.988386
25 		 0.937285
26 		 0.888476
27 		 0.842202
28 		 0.798624
29 		 0.757820
30 		 0.719792
31 		 0.684474
32 		 0.651748
33 		 0.621450
34 		 0.593395
35 		 0.567380
36 		 0.543199
37 		 0.520654
38 		 0.499558
39 		 0.479743
40 		 0.461058
41 		 0.443373
42 		 0.426582
43 		 0.410592
44 		 0.395331
45 		 0.380740
46 		 0.366773
47 		 0.353394
48 		 0.340574
49 		 0.328292
50 		 0.316529
51 		 0.305270
52 		 0.294501
53 		 0.284209
54 		 0.274380
55 		 0.265001
56 		 0.256056
57 		 0.247531
58 		 0.239407
59 		 0.231667
60 		 0.224293
61 		 0.217264
62 		 0.210560
63 		 0.204160
64 	

In [9]:
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 11.724462070909505:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


Now let's extend this to a more realistic example. Unfortunately, running this function on your entire user-movie matrix is still not something you likely want to do on your local machine.  However, we can see how well this example extends to 

`5.` Given the size of this matrix, this will take quite a bit of time.  Consider the following hyperparameters: 4 latent features, 0.005 learning rate, and 20 iterations.  Grab a snack, take a walk, and this should be done running in a bit.

In [10]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 23.263426
2 		 10.789859
3 		 7.407872
4 		 5.713551
5 		 4.644240
6 		 3.895652
7 		 3.338143
8 		 2.905617
9 		 2.560526
10 		 2.279704
11 		 2.047877
12 		 1.854439
13 		 1.691690
14 		 1.553821
15 		 1.436324
16 		 1.335626
17 		 1.248867
18 		 1.173733
19 		 1.108342
20 		 1.051149


`6.` Now that you have a set of predictions for each user-movie pair.  Let's answer a few questions about your results. Provide the correct values to each of the variables below, and check your solutions using the tests below.

In [11]:
# How many actual ratings exist in first_1000_users
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 21581148


In [12]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

AssertionError: Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right.